In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Helper libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
import io
from sklearn.utils import shuffle

# load mostly preprocessed data (preprocessing accomplished before 3/25 proposal)

# would use regular numpy syntax to load locally:
#X = np.load('<path>/X_cleaned_attempt_new.npy',allow_pickle=True)
#Y = np.load('<path>/Y_cleaned_attempt_new.npy',allow_pickle=True)
#X = np.asarray(X).astype('float32')
#Y = np.asarray(Y).astype('float32')

# i'm using colab, so it's slightly more complicated
ytrainval_byteseq = files.upload() # choose 'Y_cleaned_attempt_new.npy' from local system
ytrainval_filelike = io.BytesIO(ytrainval_byteseq['Y_cleaned_attempt_new.npy']) # create file-like object
ytrainval = np.load(ytrainval_filelike,allow_pickle=True) # create regular numpy array
xtrainval_byteseq = files.upload() # choose 'X_cleaned_attempt_new.npy' from local system
xtrainval_filelike = io.BytesIO(xtrainval_byteseq['X_cleaned_attempt_new.npy']) # create file-like object
xtrainval = np.load(xtrainval_filelike,allow_pickle=True) # create regular numpy array
ytest_byteseq = files.upload() # choose 'Y_test.npy' from local system
ytest_filelike = io.BytesIO(ytest_byteseq['Y_test.npy']) # create file-like object
ytest = np.load(ytest_filelike,allow_pickle=True) # create regular numpy array
xtest_byteseq = files.upload() # choose 'X_test.npy' from local system
xtest_filelike = io.BytesIO(xtest_byteseq['X_test.npy']) # create file-like object
xtest = np.load(xtest_filelike,allow_pickle=True) # create regular numpy array

#check shapes
print(xtrainval.shape)
print(ytrainval.shape)
print(xtest.shape)
print(ytest.shape)

#ytrainval needs to be converted to number representation of labels
feature_map = {'Rock':0,'Electronic':1,'Experimental':2,'Hip-Hop':3,'Folk':4,'Instrumental':5,'Pop':6,
              'International':7,'Classical':8,'Old-Time / Historic':9, 'Jazz':10,'Country':11,'Soul-RnB':12,
              'Spoken':13,'Blues':14,'Easy Listening':15}
for i in range(ytrainval.shape[0]):
    ytrainval[i] = float(feature_map[ytrainval[i]])
ytrainval[i] = np.asarray(ytrainval[i]).astype('float32')

print(ytrainval)
print(ytest)

# for the preliminary report, i did a roughly 80-20 split, but used the 20% as the training set to improve speed
# so will recombine and do a 90-10 split, using the 90% as training/validation
xfull = np.concatenate((xtrainval,xtest),axis=0)
yfull = np.concatenate((ytrainval,ytest),axis=0)
print(xfull.shape)
print(yfull.shape)



Saving Y_cleaned_attempt_new.npy to Y_cleaned_attempt_new.npy


Saving X_cleaned_attempt_new.npy to X_cleaned_attempt_new.npy


Saving Y_test.npy to Y_test.npy


Saving X_test.npy to X_test.npy
(9349, 520)
(9349,)
(40249, 520)
(40249,)
[3.0 3.0 3.0 ... 4.0 0.0 array(0., dtype=float32)]
[3.0 6.0 0.0 ... 0.0 0.0 array(0., dtype=float32)]
(49598, 520)
(49598,)


In [2]:
# this is where the strategy changes. We need to do a different train/test/val split
# let's see what the overall distribution between the categories is
yfull_1hot = to_categorical(yfull)
yfull_sum = np.sum(yfull_1hot, axis=0)
print(yfull_sum)


[14182.  9372. 10608.  3552.  2803.  2079.  2332.  1389.  1230.   554.
   571.   194.   175.   423.   110.    24.]


In [3]:
# In an extreme case, we might just take 20 of each class in the train set

yfull = yfull.reshape(yfull.shape[0],1)
both_full = np.concatenate((xfull, yfull), axis=1)
print(both_full.shape)
both_full = shuffle(both_full)
indices = [] 
num_counter = []
for i in range(16):
    num_counter.append(0)
train_max_list = [1000,1000,1000,1000,1000,1000,1000,1000,1000,100,100,100,100,100,100,20]
for i in range(both_full.shape[0]):
    if num_counter[int(both_full[i,-1])] < train_max_list[int(both_full[i,-1])]:
        num_counter[int(both_full[i,-1])] += 1
        indices.append(i)
print(num_counter)
both_train_small = both_full[indices]
y_train_small = both_train_small[:,-1]
y_train_small_1hot = to_categorical(y_train_small).astype('float32')
x_train_small = both_train_small[:,:-1]
print(y_train_small_1hot.shape)
print(y_train_small.shape)
print(x_train_small.shape)

(49598, 521)
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 100, 100, 100, 100, 100, 100, 20]
(9620, 16)
(9620,)
(9620, 520)


In [ ]:
# need to implement k-fold cross validation
'''
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(
        learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
        metrics=['accuracy'])


# see https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits = x_trainval_small.shape[0])
for train_index, val_index in kf.split(np.zeros(x_trainval_small.shape[0]),y_trainval_small_1hot):
	
	
	
	
	# FIT THE MODEL
	history = model.fit(x_trainval_small,
			    epochs=10,,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
	model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])
	
	tf.keras.backend.clear_session()
	
	fold_var += 1
  '''

In [ ]:
'''
# let's give the datasets better names to reflect what i'm actually doing (kind of abandoned above plan)
import copy 
x_trainval_small = x_trainval_small.astype('float32')
y_trainval_small_1hot = y_trainval_small_1hot.astype('float32')
x_train_small = copy.deepcopy(x_trainval_small)
y_train_small_1hot = copy.deepcopy(y_trainval_small_1hot)
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_trainval_small, y_trainval_small_1hot, batch_size=32, epochs=100, validation_data=(x_trainval_small, y_trainval_small_1hot))
'''

Epoch 1/100
10/10 [==============================] - 1s 37ms/step - loss: 2.9122 - accuracy: 0.1406 - val_loss: 21.3840 - val_accuracy: 0.0719
Epoch 2/100
10/10 [==============================] - 0s 13ms/step - loss: 2.0922 - accuracy: 0.3375 - val_loss: 19.8783 - val_accuracy: 0.0625
Epoch 3/100
10/10 [==============================] - 0s 13ms/step - loss: 1.7584 - accuracy: 0.4719 - val_loss: 22.5311 - val_accuracy: 0.0625
Epoch 4/100
10/10 [==============================] - 0s 9ms/step - loss: 1.4792 - accuracy: 0.5562 - val_loss: 22.1548 - val_accuracy: 0.0625
Epoch 5/100
10/10 [==============================] - 0s 14ms/step - loss: 1.2724 - accuracy: 0.6094 - val_loss: 23.0078 - val_accuracy: 0.0625
Epoch 6/100
10/10 [==============================] - 0s 10ms/step - loss: 1.1068 - accuracy: 0.6875 - val_loss: 19.2610 - val_accuracy: 0.0625
Epoch 7/100
10/10 [==============================] - 0s 10ms/step - loss: 0.9062 - accuracy: 0.7437 - val_loss: 18.2128 - val_accuracy: 0.0625


In [ ]:
'''
y_test = np.array([both_full[i,-1] for i in range(2000) if i not in indices])
x_test = np.array([both_full[i,:-1] for i in range(2000) if i not in indices])
y_test_1hot = to_categorical(y_test)
ytrain_sum = np.sum(y_train_small_1hot, axis=0)
print(y_test_1hot.shape)
print(x_test.shape)
ytest_sum = np.sum(y_test_1hot, axis=0)
print(ytest_sum)
print(yfull_sum)
print(ytrain_sum)
'''

(1747, 11)
(1747, 520)
[553. 382. 386. 119.  85.  67.  79.  24.  34.  12.   6.]
[14182.  9372. 10608.  3552.  2803.  2079.  2332.  1389.  1230.   554.
   571.   194.   175.   423.   110.    24.]
[20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [4]:
val_numbers = [(i-train_max_list[c])//2 for c,i in enumerate(yfull_sum)] # number of each instance to put in val set 
val_indices = []
num_counter = []
for i in range(16):
    num_counter.append(0)
for i in range(both_full.shape[0]):
    if i not in indices: # don't count if already in training set
        if num_counter[int(both_full[i,-1])] < val_numbers[int(both_full[i,-1])]:
            num_counter[int(both_full[i,-1])] += 1
            val_indices.append(i)

#indices and val_indices should be different lists
for i in indices:
  for j in val_indices:
    assert i!=j

both_val_small = both_full[val_indices,:]
y_val_small = both_val_small[:,-1]
y_val_small_1hot = to_categorical(y_val_small).astype('float32')
x_val_small = both_val_small[:,:-1]
print(y_val_small_1hot.shape)
print(y_val_small.shape)
print(x_val_small.shape)

(19986, 16)
(19986,)
(19986, 520)


In [5]:
print(num_counter)

[6591, 4186, 4804, 1276, 901, 539, 666, 194, 115, 227, 235, 47, 37, 161, 5, 2]


In [6]:
# now we need to put everything else in the test set
test_indices = []
for i in range(both_full.shape[0]):
    if i not in indices: # don't count if already in training set
        if i not in val_indices: # don't count if already in validation set
            test_indices.append(i)
both_test_small = both_full[test_indices,:]
y_test_small = both_test_small[:,-1]
y_test_small_1hot = to_categorical(y_test_small).astype('float32')
x_test_small = both_test_small[:,:-1]
print(y_test_small_1hot.shape)
print(y_test_small.shape)
print(x_test_small.shape)
assert y_test_small.shape[0] + y_val_small.shape[0] + y_train_small.shape[0] == both_full.shape[0]

(19992, 16)
(19992,)
(19992, 520)


In [7]:
#from copy import deepcopy
#y_train_small_1hot_copy = deepcopy(y_train_small_1hot) float32 copy
#y_val_small_1hot_copy = deepcopy(y_val_small_1hot) float32 copy
#y_test_small_1hot_copy = deepcopy(y_test_small_1hot) float32 copy
#x_train_small_copy = deepcopy(x_train_small) # float copy
#x_val_small_copy = deepcopy(x_val_small) # float copy
#x_test_small_copy = deepcopy(x_test_small) # float copy

x_train_small = x_train_small.astype('float32')
x_val_small = x_val_small.astype('float32')
x_test_small = x_test_small.astype('float32')

y_train_small_1hot = y_train_small_1hot.astype('int')
y_val_small_1hot = y_val_small_1hot.astype('int')
y_test_small_1hot = y_test_small_1hot.astype('int')

In [9]:
# now let's train!
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.3),
          layers.Dense(100,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 3s 8ms/step - loss: 1.8233 - accuracy: 0.4257 - val_loss: 2.9661 - val_accuracy: 0.0697
Epoch 2/100
301/301 [==============================] - 2s 7ms/step - loss: 1.5384 - accuracy: 0.5029 - val_loss: 1.5223 - val_accuracy: 0.5067
Epoch 3/100
301/301 [==============================] - 2s 7ms/step - loss: 1.4657 - accuracy: 0.5232 - val_loss: 1.3658 - val_accuracy: 0.5642
Epoch 4/100
301/301 [==============================] - 2s 8ms/step - loss: 1.4000 - accuracy: 0.5471 - val_loss: 1.5150 - val_accuracy: 0.5067
Epoch 5/100
301/301 [==============================] - 2s 7ms/step - loss: 1.3407 - accuracy: 0.5588 - val_loss: 1.4224 - val_accuracy: 0.5406
Epoch 6/100
301/301 [==============================] - 2s 7ms/step - loss: 1.3154 - accuracy: 0.5684 - val_loss: 1.4927 - val_accuracy: 0.5197
Epoch 7/100
301/301 [==============================] - 2s 7ms/step - loss: 1.2760 - accuracy: 0.5822 - val_loss: 1.3290 - val_accuracy: 0.5739

In [10]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          #layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=10, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/10
301/301 [==============================] - 4s 8ms/step - loss: 1.9173 - accuracy: 0.4063 - val_loss: 2.2535 - val_accuracy: 0.3110
Epoch 2/10
301/301 [==============================] - 2s 8ms/step - loss: 1.5435 - accuracy: 0.5089 - val_loss: 1.4879 - val_accuracy: 0.5375
Epoch 3/10
301/301 [==============================] - 2s 8ms/step - loss: 1.4410 - accuracy: 0.5402 - val_loss: 1.4222 - val_accuracy: 0.5583
Epoch 4/10
301/301 [==============================] - 2s 8ms/step - loss: 1.3812 - accuracy: 0.5553 - val_loss: 1.3982 - val_accuracy: 0.5727
Epoch 5/10
301/301 [==============================] - 2s 8ms/step - loss: 1.3481 - accuracy: 0.5705 - val_loss: 1.4050 - val_accuracy: 0.5660
Epoch 6/10
301/301 [==============================] - 2s 8ms/step - loss: 1.3311 - accuracy: 0.5664 - val_loss: 1.4290 - val_accuracy: 0.5497
Epoch 7/10
301/301 [==============================] - 2s 8ms/step - loss: 1.3034 - accuracy: 0.5791 - val_loss: 1.4087 - val_accuracy: 0.5561
Epoch 

In [11]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=200, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/200
301/301 [==============================] - 3s 8ms/step - loss: 2.0008 - accuracy: 0.3985 - val_loss: 5.3959 - val_accuracy: 0.0290
Epoch 2/200
301/301 [==============================] - 2s 8ms/step - loss: 1.6014 - accuracy: 0.4995 - val_loss: 1.5146 - val_accuracy: 0.5485
Epoch 3/200
301/301 [==============================] - 2s 8ms/step - loss: 1.4771 - accuracy: 0.5347 - val_loss: 1.4771 - val_accuracy: 0.5510
Epoch 4/200
301/301 [==============================] - 2s 8ms/step - loss: 1.4077 - accuracy: 0.5568 - val_loss: 1.4125 - val_accuracy: 0.5773
Epoch 5/200
301/301 [==============================] - 2s 8ms/step - loss: 1.3845 - accuracy: 0.5639 - val_loss: 1.4821 - val_accuracy: 0.5394
Epoch 6/200
301/301 [==============================] - 2s 8ms/step - loss: 1.3528 - accuracy: 0.5710 - val_loss: 1.4361 - val_accuracy: 0.5610
Epoch 7/200
301/301 [==============================] - 2s 8ms/step - loss: 1.3294 - accuracy: 0.5735 - val_loss: 1.4488 - val_accuracy: 0.5538

In [12]:
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=200, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/200
301/301 [==============================] - 3s 8ms/step - loss: 2.2433 - accuracy: 0.3251 - val_loss: 3.9729 - val_accuracy: 0.0607
Epoch 2/200
301/301 [==============================] - 2s 7ms/step - loss: 1.8108 - accuracy: 0.4452 - val_loss: 1.6598 - val_accuracy: 0.5053
Epoch 3/200
301/301 [==============================] - 2s 7ms/step - loss: 1.6657 - accuracy: 0.4802 - val_loss: 1.5559 - val_accuracy: 0.5199
Epoch 4/200
301/301 [==============================] - 2s 7ms/step - loss: 1.6015 - accuracy: 0.4943 - val_loss: 1.4788 - val_accuracy: 0.5499
Epoch 5/200
301/301 [==============================] - 2s 8ms/step - loss: 1.5610 - accuracy: 0.5060 - val_loss: 1.4376 - val_accuracy: 0.5594
Epoch 6/200
301/301 [==============================] - 2s 8ms/step - loss: 1.5520 - accuracy: 0.5082 - val_loss: 1.4373 - val_accuracy: 0.5554
Epoch 7/200
301/301 [==============================] - 2s 7ms/step - loss: 1.5282 - accuracy: 0.5187 - val_loss: 1.4270 - val_accuracy: 0.5567

In [13]:
# the previous model is starting to look like a real neural network, but one that's overfitting
# maybe more complexity could also help it get toward 100% also, but overfitting is the main concern
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 4s 9ms/step - loss: 14.2773 - accuracy: 0.3241 - val_loss: 4.5125 - val_accuracy: 0.0301
Epoch 2/100
301/301 [==============================] - 2s 8ms/step - loss: 3.6859 - accuracy: 0.3810 - val_loss: 3.5226 - val_accuracy: 0.3690
Epoch 3/100
301/301 [==============================] - 2s 8ms/step - loss: 3.4205 - accuracy: 0.4102 - val_loss: 3.2661 - val_accuracy: 0.4357
Epoch 4/100
301/301 [==============================] - 3s 8ms/step - loss: 3.2009 - accuracy: 0.4407 - val_loss: 3.0143 - val_accuracy: 0.4581
Epoch 5/100
301/301 [==============================] - 2s 8ms/step - loss: 3.0635 - accuracy: 0.4562 - val_loss: 2.8303 - val_accuracy: 0.5124
Epoch 6/100
301/301 [==============================] - 2s 8ms/step - loss: 2.9203 - accuracy: 0.4682 - val_loss: 2.8386 - val_accuracy: 0.4967
Epoch 7/100
301/301 [==============================] - 2s 8ms/step - loss: 2.8021 - accuracy: 0.4780 - val_loss: 2.7403 - val_accuracy: 0.517

In [14]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 4s 9ms/step - loss: 15.2182 - accuracy: 0.3192 - val_loss: 4.8246 - val_accuracy: 0.0284
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 4.0568 - accuracy: 0.3595 - val_loss: 3.7470 - val_accuracy: 0.4569
Epoch 3/100
301/301 [==============================] - 3s 8ms/step - loss: 3.8686 - accuracy: 0.3760 - val_loss: 3.6492 - val_accuracy: 0.4313
Epoch 4/100
301/301 [==============================] - 2s 8ms/step - loss: 3.6890 - accuracy: 0.3865 - val_loss: 3.4987 - val_accuracy: 0.4091
Epoch 5/100
301/301 [==============================] - 2s 8ms/step - loss: 3.5473 - accuracy: 0.3989 - val_loss: 3.3163 - val_accuracy: 0.4542
Epoch 6/100
301/301 [==============================] - 2s 8ms/step - loss: 3.3914 - accuracy: 0.4157 - val_loss: 3.2098 - val_accuracy: 0.4839
Epoch 7/100
301/301 [==============================] - 2s 8ms/step - loss: 3.2961 - accuracy: 0.4194 - val_loss: 3.1400 - val_accuracy: 0.47

In [15]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          #layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          #layers.BatchNormalization(),
          #layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 3s 8ms/step - loss: 18.0574 - accuracy: 0.3714 - val_loss: 6.2345 - val_accuracy: 0.0717
Epoch 2/100
301/301 [==============================] - 2s 7ms/step - loss: 4.7320 - accuracy: 0.4021 - val_loss: 4.1372 - val_accuracy: 0.4564
Epoch 3/100
301/301 [==============================] - 2s 8ms/step - loss: 3.9818 - accuracy: 0.4209 - val_loss: 3.8109 - val_accuracy: 0.4463
Epoch 4/100
301/301 [==============================] - 2s 8ms/step - loss: 3.7804 - accuracy: 0.4331 - val_loss: 3.6149 - val_accuracy: 0.4914
Epoch 5/100
301/301 [==============================] - 2s 7ms/step - loss: 3.6688 - accuracy: 0.4379 - val_loss: 3.4586 - val_accuracy: 0.5213
Epoch 6/100
301/301 [==============================] - 2s 8ms/step - loss: 3.5839 - accuracy: 0.4404 - val_loss: 3.4603 - val_accuracy: 0.4900
Epoch 7/100
301/301 [==============================] - 2s 8ms/step - loss: 3.5181 - accuracy: 0.4525 - val_loss: 3.4175 - val_accuracy: 0.525

In [16]:
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 4s 9ms/step - loss: 14.4064 - accuracy: 0.2155 - val_loss: 4.0435 - val_accuracy: 0.1929
Epoch 2/100
301/301 [==============================] - 3s 9ms/step - loss: 3.7195 - accuracy: 0.2753 - val_loss: 3.5633 - val_accuracy: 0.2903
Epoch 3/100
301/301 [==============================] - 3s 9ms/step - loss: 3.5364 - accuracy: 0.2878 - val_loss: 3.3386 - val_accuracy: 0.3239
Epoch 4/100
301/301 [==============================] - 2s 8ms/step - loss: 3.3623 - accuracy: 0.2979 - val_loss: 3.2101 - val_accuracy: 0.3127
Epoch 5/100
301/301 [==============================] - 4s 13ms/step - loss: 3.2366 - accuracy: 0.3106 - val_loss: 3.2990 - val_accuracy: 0.2196
Epoch 6/100
301/301 [==============================] - 2s 8ms/step - loss: 3.1231 - accuracy: 0.3393 - val_loss: 3.0110 - val_accuracy: 0.2951
Epoch 7/100
301/301 [==============================] - 3s 9ms/step - loss: 3.0601 - accuracy: 0.3497 - val_loss: 3.0039 - val_accuracy: 0.35

In [17]:
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.25),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 5s 14ms/step - loss: 14.7420 - accuracy: 0.2396 - val_loss: 4.1664 - val_accuracy: 0.0870
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 3.8013 - accuracy: 0.2737 - val_loss: 3.4865 - val_accuracy: 0.3512
Epoch 3/100
301/301 [==============================] - 3s 9ms/step - loss: 3.6122 - accuracy: 0.2844 - val_loss: 3.3684 - val_accuracy: 0.3631
Epoch 4/100
301/301 [==============================] - 4s 13ms/step - loss: 3.4225 - accuracy: 0.3006 - val_loss: 3.2223 - val_accuracy: 0.3839
Epoch 5/100
301/301 [==============================] - 4s 13ms/step - loss: 3.2787 - accuracy: 0.3096 - val_loss: 2.9786 - val_accuracy: 0.4142
Epoch 6/100
301/301 [==============================] - 4s 13ms/step - loss: 3.1969 - accuracy: 0.3202 - val_loss: 2.8917 - val_accuracy: 0.4070
Epoch 7/100
301/301 [==============================] - 4s 13ms/step - loss: 3.0932 - accuracy: 0.3203 - val_loss: 2.8793 - val_accuracy:

In [18]:
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.25),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 5s 14ms/step - loss: 15.4253 - accuracy: 0.2785 - val_loss: 4.6458 - val_accuracy: 0.2282
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 4.1498 - accuracy: 0.3268 - val_loss: 4.0213 - val_accuracy: 0.3404
Epoch 3/100
301/301 [==============================] - 3s 9ms/step - loss: 3.9044 - accuracy: 0.3330 - val_loss: 3.6179 - val_accuracy: 0.4080
Epoch 4/100
301/301 [==============================] - 3s 9ms/step - loss: 3.7115 - accuracy: 0.3406 - val_loss: 3.4294 - val_accuracy: 0.4662
Epoch 5/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5490 - accuracy: 0.3432 - val_loss: 3.2862 - val_accuracy: 0.4359
Epoch 6/100
301/301 [==============================] - 4s 13ms/step - loss: 3.3768 - accuracy: 0.3576 - val_loss: 3.2538 - val_accuracy: 0.4473
Epoch 7/100
301/301 [==============================] - 4s 14ms/step - loss: 3.2826 - accuracy: 0.3641 - val_loss: 3.1454 - val_accuracy: 

In [19]:
'''
# val loss and train loss were pretty similar, can we add just a little more regularized complexity?
# maybe dropout makes a difference
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1',bias_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(4,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))
'''

'\n# val loss and train loss were pretty similar, can we add just a little more regularized complexity?\n# maybe dropout makes a difference\nlayer_list = [ #520 parameters in input\n          layers.BatchNormalization(),\n          layers.Dropout(0.4),\n          layers.Dense(520,kernel_initializer=\'lecun_normal\',kernel_regularizer=\'l1\',bias_regularizer=\'l1\'),\n          layers.BatchNormalization(),\n          layers.Activation(\'selu\'),\n          layers.Dense(4,kernel_initializer=\'lecun_normal\'), #let\'s try dimensionality reduction\n          layers.BatchNormalization(),\n          layers.Activation(\'selu\'),\n          layers.Dense(16,kernel_initializer=\'lecun_normal\'), \n          layers.BatchNormalization(),\n          layers.Activation(\'selu\'),\n          layers.Dense(16,activation=\'softmax\',kernel_initializer=\'lecun_normal\'),\n          ]\nmodel = Sequential(layer_list)\nmodel.compile(optimizer=keras.optimizers.Adam(\n    learning_rate=keras.optimizers.schedul

In [20]:
# further modifying the previous one that had the lowest validation loss
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
301/301 [==============================] - 3s 8ms/step - loss: 2.2214 - accuracy: 0.3266 - val_loss: 1.8802 - val_accuracy: 0.3886
Epoch 2/400
301/301 [==============================] - 2s 7ms/step - loss: 1.8050 - accuracy: 0.4469 - val_loss: 1.6293 - val_accuracy: 0.5144
Epoch 3/400
301/301 [==============================] - 2s 7ms/step - loss: 1.6661 - accuracy: 0.4802 - val_loss: 1.5500 - val_accuracy: 0.5348
Epoch 4/400
301/301 [==============================] - 2s 7ms/step - loss: 1.6114 - accuracy: 0.4980 - val_loss: 1.4935 - val_accuracy: 0.5435
Epoch 5/400
301/301 [==============================] - 2s 7ms/step - loss: 1.5682 - accuracy: 0.5049 - val_loss: 1.4675 - val_accuracy: 0.5518
Epoch 6/400
301/301 [==============================] - 2s 7ms/step - loss: 1.5322 - accuracy: 0.5125 - val_loss: 1.4595 - val_accuracy: 0.5566
Epoch 7/400
301/301 [==============================] - 2s 7ms/step - loss: 1.5244 - accuracy: 0.5107 - val_loss: 1.4397 - val_accuracy: 0.5633

In [ ]:
# further modifying the previous one that had the lowest validation loss
# clearly we need some strong regularization with this small training set
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/400
301/301 [==============================] - 3s 8ms/step - loss: 2.5032 - accuracy: 0.2223 - val_loss: 2.1556 - val_accuracy: 0.3351
Epoch 2/400
301/301 [==============================] - 2s 7ms/step - loss: 2.0937 - accuracy: 0.3292 - val_loss: 1.9349 - val_accuracy: 0.4224
Epoch 3/400
301/301 [==============================] - 2s 7ms/step - loss: 1.9647 - accuracy: 0.3647 - val_loss: 1.8265 - val_accuracy: 0.4503
Epoch 4/400
301/301 [==============================] - 2s 7ms/step - loss: 1.9095 - accuracy: 0.3770 - val_loss: 1.7826 - val_accuracy: 0.4521
Epoch 5/400
301/301 [==============================] - 2s 7ms/step - loss: 1.8729 - accuracy: 0.3860 - val_loss: 1.7195 - val_accuracy: 0.4920
Epoch 6/400
301/301 [==============================] - 2s 7ms/step - loss: 1.8538 - accuracy: 0.3957 - val_loss: 1.6985 - val_accuracy: 0.4885
Epoch 7/400
301/301 [==============================] - 2s 7ms/step - loss: 1.8328 - accuracy: 0.3953 - val_loss: 1.6858 - val_accuracy: 0.4850

In [ ]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [ ]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [ ]:
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [ ]:
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(100,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [ ]:
# that didn't help
# might need to lighten up on the dimensionality reduction
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(40,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [ ]:
# that didn't help
# might need to lighten up on the dimensionality reduction
# so I created a model that isn't learning the training set! let's add another layer
layer_list = [ #520 parameters in input
          layers.Dropout(0.6),
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(40,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal',kernel_regularizer='l2'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(0.6),
          layers.Dense(400,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))

In [8]:
# it appears that this one was on the right track but needed more complexity
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 8s 14ms/step - loss: 14.7015 - accuracy: 0.3223 - val_loss: 4.2405 - val_accuracy: 0.1912
Epoch 2/100
301/301 [==============================] - 5s 15ms/step - loss: 3.7446 - accuracy: 0.3839 - val_loss: 3.6120 - val_accuracy: 0.4471
Epoch 3/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5008 - accuracy: 0.4030 - val_loss: 3.3449 - val_accuracy: 0.4438
Epoch 4/100
301/301 [==============================] - 4s 15ms/step - loss: 3.2867 - accuracy: 0.4247 - val_loss: 3.2592 - val_accuracy: 0.3690
Epoch 5/100
301/301 [==============================] - 4s 13ms/step - loss: 3.0945 - accuracy: 0.4402 - val_loss: 2.9947 - val_accuracy: 0.4523
Epoch 6/100
301/301 [==============================] - 4s 13ms/step - loss: 2.9719 - accuracy: 0.4503 - val_loss: 2.8123 - val_accuracy: 0.5090
Epoch 7/100
301/301 [==============================] - 4s 13ms/step - loss: 2.8741 - accuracy: 0.4670 - val_loss: 2.8489 - val_accuracy

In [9]:
# adding the extra layer after dimensionality reduction might not have accomplished much?
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(8,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(160,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 7s 17ms/step - loss: 14.4475 - accuracy: 0.3763 - val_loss: 4.4700 - val_accuracy: 0.0651
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 3.7216 - accuracy: 0.3944 - val_loss: 3.4773 - val_accuracy: 0.4336
Epoch 3/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5307 - accuracy: 0.4162 - val_loss: 3.5221 - val_accuracy: 0.3458
Epoch 4/100
301/301 [==============================] - 5s 15ms/step - loss: 3.3035 - accuracy: 0.4422 - val_loss: 3.2213 - val_accuracy: 0.4779
Epoch 5/100
301/301 [==============================] - 5s 15ms/step - loss: 3.1065 - accuracy: 0.4524 - val_loss: 3.0775 - val_accuracy: 0.4522
Epoch 6/100
301/301 [==============================] - 5s 15ms/step - loss: 2.9652 - accuracy: 0.4682 - val_loss: 2.8675 - val_accuracy: 0.5027
Epoch 7/100
301/301 [==============================] - 5s 15ms/step - loss: 2.8877 - accuracy: 0.4798 - val_loss: 2.7538 - val_accuracy

In [10]:
# that probably wasn't any better. going to relax dimensionality reduction
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(160,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 5s 14ms/step - loss: 15.0015 - accuracy: 0.3835 - val_loss: 4.9873 - val_accuracy: 0.0599
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 3.9656 - accuracy: 0.4005 - val_loss: 3.9194 - val_accuracy: 0.3756
Epoch 3/100
301/301 [==============================] - 4s 13ms/step - loss: 3.7555 - accuracy: 0.4267 - val_loss: 3.6284 - val_accuracy: 0.4731
Epoch 4/100
301/301 [==============================] - 4s 12ms/step - loss: 3.5494 - accuracy: 0.4404 - val_loss: 3.4915 - val_accuracy: 0.4496
Epoch 5/100
301/301 [==============================] - 4s 13ms/step - loss: 3.3602 - accuracy: 0.4560 - val_loss: 3.2182 - val_accuracy: 0.4732
Epoch 6/100
301/301 [==============================] - 5s 15ms/step - loss: 3.2066 - accuracy: 0.4693 - val_loss: 3.0821 - val_accuracy: 0.4927
Epoch 7/100
301/301 [==============================] - 4s 13ms/step - loss: 3.0942 - accuracy: 0.4768 - val_loss: 3.0514 - val_accuracy

In [11]:
# I think we need just a little more complexity
# that probably wasn't any better. going to relax dimensionality reduction
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(160,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 6s 16ms/step - loss: 15.5771 - accuracy: 0.3910 - val_loss: 5.1314 - val_accuracy: 0.0807
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 4.1039 - accuracy: 0.4131 - val_loss: 3.9200 - val_accuracy: 0.4833
Epoch 3/100
301/301 [==============================] - 5s 15ms/step - loss: 3.8929 - accuracy: 0.4244 - val_loss: 3.8072 - val_accuracy: 0.4528
Epoch 4/100
301/301 [==============================] - 4s 13ms/step - loss: 3.7018 - accuracy: 0.4420 - val_loss: 3.4438 - val_accuracy: 0.4835
Epoch 5/100
301/301 [==============================] - 5s 15ms/step - loss: 3.5269 - accuracy: 0.4612 - val_loss: 3.3822 - val_accuracy: 0.4917
Epoch 6/100
301/301 [==============================] - 5s 15ms/step - loss: 3.4035 - accuracy: 0.4688 - val_loss: 3.4634 - val_accuracy: 0.4305
Epoch 7/100
301/301 [==============================] - 5s 15ms/step - loss: 3.2658 - accuracy: 0.4762 - val_loss: 3.2334 - val_accuracy

In [12]:
# I think we need just a little more complexity
# that probably wasn't any better. going to relax dimensionality reduction
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 5s 14ms/step - loss: 14.9923 - accuracy: 0.3677 - val_loss: 5.1408 - val_accuracy: 0.0284
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 3.9200 - accuracy: 0.4014 - val_loss: 3.7859 - val_accuracy: 0.3299
Epoch 3/100
301/301 [==============================] - 5s 15ms/step - loss: 3.6857 - accuracy: 0.4280 - val_loss: 3.5884 - val_accuracy: 0.4807
Epoch 4/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5330 - accuracy: 0.4444 - val_loss: 3.4619 - val_accuracy: 0.4488
Epoch 5/100
301/301 [==============================] - 5s 15ms/step - loss: 3.3951 - accuracy: 0.4561 - val_loss: 3.2565 - val_accuracy: 0.4947
Epoch 6/100
301/301 [==============================] - 4s 15ms/step - loss: 3.2895 - accuracy: 0.4613 - val_loss: 3.2873 - val_accuracy: 0.4899
Epoch 7/100
301/301 [==============================] - 4s 13ms/step - loss: 3.1820 - accuracy: 0.4778 - val_loss: 3.1048 - val_accuracy

In [13]:
# I think we need just a little more complexity
# that probably wasn't any better. going to relax dimensionality reduction
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 6s 16ms/step - loss: 14.6967 - accuracy: 0.3786 - val_loss: 5.2631 - val_accuracy: 0.0288
Epoch 2/100
301/301 [==============================] - 5s 15ms/step - loss: 3.9352 - accuracy: 0.4079 - val_loss: 3.8214 - val_accuracy: 0.4128
Epoch 3/100
301/301 [==============================] - 4s 12ms/step - loss: 3.6738 - accuracy: 0.4366 - val_loss: 3.5314 - val_accuracy: 0.4795
Epoch 4/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5052 - accuracy: 0.4590 - val_loss: 3.4766 - val_accuracy: 0.4291
Epoch 5/100
301/301 [==============================] - 5s 15ms/step - loss: 3.3717 - accuracy: 0.4718 - val_loss: 3.2670 - val_accuracy: 0.5191
Epoch 6/100
301/301 [==============================] - 4s 13ms/step - loss: 3.2585 - accuracy: 0.4765 - val_loss: 3.2525 - val_accuracy: 0.4868
Epoch 7/100
301/301 [==============================] - 4s 12ms/step - loss: 3.1639 - accuracy: 0.4867 - val_loss: 3.0287 - val_accuracy

In [14]:
# I think we need just a little more complexity
# that probably wasn't any better. going to relax dimensionality reduction
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          #layers.Dropout(0.4),
          layers.Dense(520,kernel_initializer='lecun_normal',kernel_regularizer='l1'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'), 
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=100, validation_data=(x_val_small, y_val_small_1hot))

Epoch 1/100
301/301 [==============================] - 6s 16ms/step - loss: 14.4877 - accuracy: 0.3789 - val_loss: 4.3712 - val_accuracy: 0.0515
Epoch 2/100
301/301 [==============================] - 4s 13ms/step - loss: 3.8286 - accuracy: 0.4095 - val_loss: 3.7049 - val_accuracy: 0.4897
Epoch 3/100
301/301 [==============================] - 4s 13ms/step - loss: 3.5973 - accuracy: 0.4362 - val_loss: 3.5144 - val_accuracy: 0.4884
Epoch 4/100
301/301 [==============================] - 4s 13ms/step - loss: 3.4461 - accuracy: 0.4585 - val_loss: 3.2868 - val_accuracy: 0.5066
Epoch 5/100
301/301 [==============================] - 4s 15ms/step - loss: 3.3230 - accuracy: 0.4756 - val_loss: 3.1000 - val_accuracy: 0.5170
Epoch 6/100
301/301 [==============================] - 5s 15ms/step - loss: 3.1762 - accuracy: 0.4795 - val_loss: 3.2357 - val_accuracy: 0.4526
Epoch 7/100
301/301 [==============================] - 4s 15ms/step - loss: 3.1094 - accuracy: 0.4881 - val_loss: 3.1304 - val_accuracy